In [1]:
import geopandas as gpd
import pandas as pd

import ee

In [2]:
ee.Initialize()

In [3]:
# Training data

from shapely.geometry import Point

training  = gpd.GeoDataFrame({
    'class_name': ['class_1', 'class_2', 'class_3'],
    'geometry': [Point(0, 0), Point(1, 1), Point(2, 2)]
})

testing = gpd.GeoDataFrame({
    'class_name': ['class_1', 'class_2', 'class_3'],
    'geometry': [Point(0, 1), Point(1, 2), Point(2, 3)]
})


In [4]:
from cnwi.cnwilib.features import insert_is_training

gdf, lookup = insert_is_training(training, testing)

In [9]:
# create a data lookup
from cnwi.cnwilib.features import create_data_lookup
data_lookup = create_data_lookup(gdf['class_name'])
data_lookup

,class_name,value
0,class_1,1
1,class_2,2
2,class_3,3


In [10]:
from cnwi.cnwilib.features import insert_values_into_features
gdf = insert_values_into_features(gdf, data_lookup)
gdf

,class_name,geometry,is_training,value
0,class_1,POINT (0.00000 0.00000),1,1
1,class_1,POINT (0.00000 1.00000),0,1
2,class_2,POINT (1.00000 1.00000),1,2
3,class_2,POINT (1.00000 2.00000),0,2
4,class_3,POINT (2.00000 2.00000),1,3
5,class_3,POINT (2.00000 3.00000),0,3


In [12]:
# convert to ee.FeatureCollection
from pprint import pprint
fc = ee.FeatureCollection(gdf.__geo_interface__)
pprint(fc.first().getInfo())

{'geometry': {'coordinates': [0, 0], 'type': 'Point'},
 'id': '0',
 'properties': {'class_name': 'class_1', 'is_training': 1, 'value': 1},
 'type': 'Feature'}


In [13]:
# create our samples for training and testing
from cnwi.cnwilib.features import sample_regions

# synthetic image
projection = ee.Projection("EPSG:4326")
image = (
    ee.Image(list(range(1, 7)))
    .rename(["B1", "B2", "B3", "B4", "B5", "B6"])
    .reproject(projection)
)

samples = sample_regions(image, fc, scale=30, projection=projection)
pprint(samples.first().getInfo())


{'geometry': None,
 'id': '0_0',
 'properties': {'B1': 1,
                'B2': 2,
                'B3': 3,
                'B4': 4,
                'B5': 5,
                'B6': 6,
                'class_name': 'class_1',
                'is_training': 1,
                'value': 1},
 'type': 'Feature'}
